# Formatting Tables in Pandas


In [2]:
# Import pandas
import pandas as pd
# Import numpy
import numpy as np



## The data
Let's create some simulated data for two widgets, A and B. We'll create a dataframe for each widget, then concatenate them together. We'll also sort the dataframe by month and reset the index.

In [3]:
# simulated data for widget A
df_a = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 1_000_000,
            high = 2_500_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 300_000,
            high = 500_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 750_000,
            high = 1_250_000,
            size = 132
        )
    }
)

df_a['Product'] = 'A'

# simulated data for widget B
df_b = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 100_000,
            high = 800_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 10_000,
            high = 95_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 450_000,
            high = 750_000,
            size = 132
        )
    }
)

df_b['Product'] = 'B'

# put it together & sort
df = pd.concat([df_a,df_b],axis = 0)
df.sort_values(by = 'Month',inplace = True)
df.reset_index(drop = True,inplace = True)

<ipython-input-3-92d32091f486>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  'Month':pd.date_range(
<ipython-input-3-92d32091f486>:32: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  'Month':pd.date_range(


Let’s calculate a few “interesting” statistics — average sale amounts and product conversion:

In [4]:
# average sale
df['Average sale'] = df['Amounts'] / df['Numbers'].replace({0: np.nan})

# conversion
df['Product conversion'] = df['Numbers'] / df['Quotes'].replace({0: np.nan})

In [5]:
df.head(3)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01-01,1975087,454992,1138182,A,2.501543,0.230366
1,2012-01-01,185836,49721,644193,B,12.956155,0.267553
2,2012-02-01,1871374,413039,1015107,A,2.457654,0.220714


from matplotlib import pyplot as plt
_df_0['Quotes'].plot(kind='hist', bins=20, title='Quotes')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Numbers'].plot(kind='hist', bins=20, title='Numbers')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Amounts'].plot(kind='hist', bins=20, title='Amounts')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Average sale'].plot(kind='hist', bins=20, title='Average sale')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_16['Average sale'].plot(kind='line', figsize=(8, 4), title='Average sale')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['Product'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='Quotes', y='Product', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['Product'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='Numbers', y='Product', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['Product'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='Amounts', y='Product', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['Product'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='Average sale', y='Product', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

## Date Formatting
There’s arguably nothing __wrong__ with the formatting, but it could be better. For instance, since all the monthly data is reflected as at the first of each month, there’s probably little sense in keeping the day element of each Month entry as it tells the reader very little.

In [6]:
# format the date as YYYY-MM
styler = df.iloc[:3].style.format({'Month':'{:%Y-%m}'})
if styler:
    display(styler)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01,1975087,454992,1138182,A,2.501543,0.230366
1,2012-01,185836,49721,644193,B,12.956155,0.267553
2,2012-02,1871374,413039,1015107,A,2.457654,0.220714


Now, we can improve readability even further by using the name of each month rather than the month number, and we can do this __*without having to alter the underlying data*__.

In [7]:
styler = df.iloc[:3].style.format({'Month':'{:%B %Y}'})
if styler:
    display(styler)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,January 2012,1975087,454992,1138182,A,2.501543,0.230366
1,January 2012,185836,49721,644193,B,12.956155,0.267553
2,February 2012,1871374,413039,1015107,A,2.457654,0.220714


Maybe a little too wordy now — let’s use abbreviations instead (e.g. “Jan” instead of “January”) and we’ll also add a comma before the year.

In [8]:
styler = df.iloc[:3].style.format({'Month':'{:%b, %Y}'})
if styler:
    display(styler)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,"Jan, 2012",2369439,372465,917247,A,2.462639,0.157195
1,"Jan, 2012",730816,33729,671857,B,19.919268,0.046153
2,"Feb, 2012",1653848,490166,1105553,A,2.255467,0.296379


## Formatting numbers with a thousand separator
A fairly straightforward formatting experience here as we separate thousands of Quotes and Numbers using commas.

What is important to note however, is that if we also want to retain the formatting we applied to the Month column (we do), then we need to extend the formatting dictionary.


In [8]:
styler = df.iloc[:3].style.format(
    {
        'Month':'{:%b, %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}'
    }
)
if styler:
    display(styler)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,"Jan, 2012","1,975,087","454,992",1138182,A,2.501543,0.230366
1,"Jan, 2012","185,836","49,721",644193,B,12.956155,0.267553
2,"Feb, 2012","1,871,374","413,039",1015107,A,2.457654,0.220714


## Formatting currencies
The Widget Company just so happens to produce and sell its widgets in a country that uses a currency denoted by £ (I hope somewhere warmer and sunnier than the country where I earn my £).

Let’s reflect that in the table, reminding ourselves that:

- At an overall level, using decimal points is probably a little much
- At a lower level — say for instance, the average sale value — using decimals can be useful.
So we add currency formatting for Amounts and Average sale to our formatting dictionary:

In [10]:
styler = df.iloc[:3].style.format(
    {
        'Month':'{:%b, %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'£{:,.0f}',
        'Average sale':'£{:,.2f}'
    }
)
if styler:
    display(styler)


# breakpoint()

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,"Jan, 2012","1,975,087","454,992","£1,138,182",A,£2.50,0.230366
1,"Jan, 2012","185,836","49,721","£644,193",B,£12.96,0.267553
2,"Feb, 2012","1,871,374","413,039","£1,015,107",A,£2.46,0.220714


## Formatting percentages
Another fairly straightforward formatting step, it’s much easier to view ratios when they are expressed as percentages rather than floating point numbers.

We’ll add Product conversion to our formatting dictionary:

In [11]:
styler = df.iloc[:3].style.format(
    {
        'Month':'{:%b, %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'£{:,.0f}',
        'Average sale':'£{:,.2f}',
        'Product conversion':'{:.2%}'
    }
)
if styler:
    display(styler)

# breakpoint()

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,"Jan, 2012","1,975,087","454,992","£1,138,182",A,£2.50,23.04%
1,"Jan, 2012","185,836","49,721","£644,193",B,£12.96,26.76%
2,"Feb, 2012","1,871,374","413,039","£1,015,107",A,£2.46,22.07%


## Hiding Indexes
The index is a useful thing to have, but it’s not always necessary to display it. In this case, we can hide it by setting index = False:


In [1]:
styler = df.iloc[:3].style.format(
    {
        'Month':'{:%b, %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'£{:,.0f}',
        'Average sale':'£{:,.2f}',
        'Product conversion':'{:.2%}'
    }
).hide()  # hide the index

if styler:
    display(styler)

breakpoint()


NameError: ignored

## Conditional Formatting
Conditional formatting is a great way to highlight certain values in a table. For instance, we might want to highlight the highest and lowest values in each column.
Let's start with highlighting rows if an element of the row meets a given condition — in this case, highlight all rows containing information relating to product A.

We do this in two steps:

1. Define the function highlight_product which returns a string if the given condition is met (that is, if the row relates to the specified product). The string contains a format command that we will pass through to the Styler.
2. The resulting format command is fed through using the apply command.



In [ ]:
def highlight_product(row, product, background_color='yellow'):
    if row['Product'] == product:
        return len(row) * [f'background-color: {background_color}']
    else:
        return len(row) * ['']

styler = df.iloc[:3].style\
    .apply(  # apply the conditional formatting
        highlight_product,
        product = 'A',
        axis = 1
    ).format(  # format the table
        {
            'Month':'{:%b, %Y}',
            'Quotes':'{:,.0f}',
            'Numbers':'{:,.0f}',
            'Amounts':'£{:,.0f}',
            'Average sale':'£{:,.2f}',
            'Product conversion':'{:.2%}'
        }
    ).hide()  # hide the index

if styler:
    display(styler)

breakpoint()

## Highlighting the highest and lowest values in each column
We can also highlight the highest and lowest values in each column. We do this in two steps:
1. Define the function highlight_min_max which returns a string if the given condition is met (that is, if the value is the highest or lowest in the column). The string contains a format command that we will pass through to the Styler.
2. The resulting format command is fed through using the applymap command.
3. We also add a gradient to the table using the background_gradient command.
4. Finally, we add a caption to the table using the set_caption command.
5. We can also add a title to the table using the set_table_styles command.   

In [ ]:
def highlight_min_max(value, background_color='yellow'):
    breakpoint()
    # ret = ''
    if value == value.min():
        ret = f'background-color: {background_color}'
    elif value == value.max():
        ret = f'background-color: {background_color}'
    else:
        ret = ''
    print(ret)
    return ret

styler = df.style\
    .highlight_min(color='pink')\
    .highlight_max(color='lightgreen')\
    .format(  # format the table
        {
            'Month':'{:%b, %Y}',
            'Quotes':'{:,.0f}',
            'Numbers':'{:,.0f}',
            'Amounts':'£{:,.0f}',
            'Average sale':'£{:,.2f}',
            'Product conversion':'{:.2%}'
        }
    ).hide()  # hide the index

breakpoint()

